#### Simple Gen AI App Using LangChain

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
# Langsmith Tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [ ]:
# Data Ingestion
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    "https://docs.smith.langchain.com/tutorials/Administrators/manage_spend"
)
loader

In [ ]:
docs = loader.load()
docs

In [ ]:
# Load Data -> Docs -> Divide documents into chunk documents -> Text -> Vectors -> Vector Embeddings -> Vector Store DB 
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [ ]:
documents

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [ ]:
vectorstoredb

In [ ]:
# Query from a vector db
query = 'Langsmith has two usage limits: total traces and extended'
result = vectorstoredb.similarity_search(query)
result[0].page_content

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')

In [ ]:
# Retrieval chain, document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """
)
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

In [ ]:
from langchain_core.documents import Document
document_chain.invoke(
    {
        "input": "Langsmith has two usage limits: total traces and extended",
        "context": [
            Document(
                page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. "
            )
        ],
    }
)

In [ ]:
# Input -> Retriever -> Vector Store DB
vectorstoredb

In [ ]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
retrieval_chain

In [ ]:
# Get the response from LLM
response = retrieval_chain.invoke(
    {"input": "LangSmith has two usage limits: total traces and extended"}
)
response['answer']

In [ ]:
response

In [ ]:
response['context']